In [68]:
import os
import json
from collections import defaultdict
import re

def process_race_file(race_file):
    """Extract driver data from a single race JSON file and update the career results dictionary."""
    with open(race_file, 'r', encoding='utf-8') as file:
        race_data = json.load(file)

    race_name = race_data.get("race_name", "Unknown Race")
    race_date = race_data.get("date", "Unknown Date")
    race_results = race_data.get("results", {}).get("race", [])
    quali_results = race_data.get("results", {}).get("qualifying", [])
    fp1_results = race_data.get("results", {}).get("free_practice_1", [])
    fp2_results = race_data.get("results", {}).get("free_practice_2", [])
    fp3_results = race_data.get("results", {}).get("free_practice_3", [])
    fastest_laps = race_data.get("results", {}).get("fastest_laps", [])
    dotd = race_data.get("results", {}).get("driver_of_the_day", [])
    driver_standings = race_data.get("results", {}).get("driver", [])
    constructor_standings_and_past_race_winners = race_data.get("results", {}).get("constructor", [])


    drivers = []

    for entry in race_results:
        driver_name = entry["Driver"]
        driver_name_proc = driver_name.replace("#", "")
        driver_name_proc = re.sub(r'\d+', '', driver_name_proc)
        drivers.append(driver_name_proc)

        #create a json file for each driver
        if not os.path.exists(f"C:/Users/athen/Desktop/Github/BestF1DriverEverWithData/data/f1_driver_careers/{driver_name_proc}/"):
            os.mkdir(f"C:/Users/athen/Desktop/Github/BestF1DriverEverWithData/data/f1_driver_careers/{driver_name_proc}")
            #create a json file for each driver
            with open(f"C:/Users/athen/Desktop/Github/BestF1DriverEverWithData/data/f1_driver_careers/{driver_name_proc}/{driver_name_proc}.json", 'w', encoding='utf-8') as file:
                json.dump({"driver_name": driver_name_proc}, file, indent=4)

        
        driver_team = entry["Constructor"]
        race_position = entry.get("Pos.", "Unknown")
        quali_entry = [q for q in quali_results if q["Driver"] == driver_name]
        quali_position = quali_entry[0].get("Pos.", "Unknown") if quali_entry else "Unknown"
        fp1_entry = [fp for fp in fp1_results if fp["Driver"] == driver_name]
        fp1_position = fp1_entry[0].get("Pos.", "Unknown") if fp1_entry else "Unknown"
        fp2_entry = [fp for fp in fp2_results if fp["Driver"] == driver_name]
        fp2_position = fp2_entry[0].get("Pos.", "Unknown") if fp2_entry else "Unknown"
        fp3_entry = [fp for fp in fp3_results if fp["Driver"] == driver_name]
        fp3_position = fp3_entry[0].get("Pos.", "Unknown") if fp3_entry else "Unknown"
        fastest_lap_entry = [fl for fl in fastest_laps if fl["Driver"] == driver_name]
        fastest_lap_position = fastest_lap_entry[0].get("Pos.", "Unknown") if fastest_lap_entry else "Unknown"
        dotd_entry = [d for d in dotd if 'Driver' in d and d["Driver"] == driver_name]
        dotd_position = dotd_entry[0].get("Pos.", "Unknown") if dotd_entry else "Unknown"
        driver_standings_entry = [ds for ds in driver_standings if ds["Driver"] == driver_name_proc]
        driver_standings_position = driver_standings_entry[0].get("Pos.", "Unknown") if driver_standings_entry else "Unknown"
        constructor_standings = [cs for cs in constructor_standings_and_past_race_winners if 'Constructor' in cs]
        constructor_standings_entry = [cs for cs in constructor_standings if cs["Constructor"] == driver_team]
        constructor_standings_position = constructor_standings_entry[0].get("Pos.", "Unknown") if constructor_standings_entry else "Unknown"


        if race_position in ['DNF', 'NC']:
            race_position = 20
        elif quali_position in ['DNQ','NC']:
            quali_position = 20



        rise_fall_factor = (int(quali_position) - int(race_position))

        with open(f"C:/Users/athen/Desktop/Github/BestF1DriverEverWithData/data/f1_driver_careers/{driver_name_proc}/{driver_name_proc}.json", 'r', encoding='utf-8') as file:
            driver_career_data = json.load(file)
        
        


        teammate_race_entry = [t for t in race_results if driver_team == t['Constructor'] and t["Driver"] != driver_name]
        teammate_name = teammate_race_entry[0]["Driver"] if teammate_race_entry else "Unknown"
        teammate_comp_factor_race = int(race_position) - int(teammate_race_entry[0]["Pos."]) if teammate_race_entry and teammate_race_entry[0]["Pos."] not in ['DNF', 'NC'] else -(20-int(race_position))
        teammate_quali_entry = [t for t in quali_results if driver_team == t['Constructor'] and t["Driver"] != driver_name]
        teammate_comp_factor_quali = int(quali_position) - int(teammate_quali_entry[0]["Pos."]) if teammate_quali_entry and teammate_quali_entry[0]["Pos."] not in ['DNQ','NC'] else -(20-int(quali_position))



        driver_career_data[race_name] = {
            "race_position": race_position,
            "quali_position": quali_position,
            "fp1_position": fp1_position,
            "fp2_position": fp2_position,
            "fp3_position": fp3_position,
            "fastest_lap_position": fastest_lap_position,
            "dotd_position": dotd_position,
            "driver_standings_position": driver_standings_position,
            "constructor_standings_position": constructor_standings_position,
            "rise_fall_factor": rise_fall_factor,
            "teammate": teammate_name,
            "teammate_comp_race": teammate_comp_factor_race,
            "teammate_comp_quali": teammate_comp_factor_quali
        }

        with open(f"C:/Users/athen/Desktop/Github/BestF1DriverEverWithData/data/f1_driver_careers/{driver_name_proc}/{driver_name_proc}.json", 'w', encoding='utf-8') as file:
            json.dump(driver_career_data, file, indent=4)




samp = ["C:/Users/athen/Desktop/Github/BestF1DriverEverWithData/data/f1_all_seasons_results/2024/2024 Abu Dhabi Grand Prix.json","C:/Users/athen/Desktop/Github/BestF1DriverEverWithData/data/f1_all_seasons_results/2024/2024 Emilia Romagna Grand Prix.json"]

for i in samp:
    process_race_file(i)